In [1]:
import os
print(f"Current working folder: {os.getcwd()}")

os.chdir('D:/My Document/Khóa Luận Tốt Nghiệp/Model Reposity/RAG_LLM_DA')
print(f"Current working folder: {os.getcwd()}")

from stages.stage_1_learn_rules_from_data.data_loader import DataLoader
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from openai_llm.llm_init import LLM_Model
from utils import load_json_data, save_json_data, load_vectorstore_db, calculate_similarity, lookup_vector_db

Current working folder: d:\My Document\Khóa Luận Tốt Nghiệp\Model Reposity\RAG_LLM_DA\notebooks
Current working folder: D:\My Document\Khóa Luận Tốt Nghiệp\Model Reposity\RAG_LLM_DA


In [2]:
llm_instance = LLM_Model()

Loading data from config/llm_config.json


In [3]:
data_dict = load_vectorstore_db(llm_instance, 'YAGO')
for collection in data_dict:
    print(f"{collection}: {len(data_dict[collection]['vector_db'].get()['documents'])} documents")

Loading data from config/data_embedding.json
facts: 544212 documents
rules: 0 documents


In [4]:


# filter = {"$and": [
# 	{"object": {"$in": ["Malaysia", 'China']}},
# 	{"subject": {"$in": ["Malaysia", 'China']}},
#     # {"relation": "Express_intent_to_cooperate"}
# ]
# }

# filter_ = {"$or":[{"object": "Communes_In_France"}, {"subject": "Balleroy"}]}
filter_ = {'subject_id': 26}
docs = lookup_vector_db("", filter_, data_dict['facts']['vector_db'], llm_instance, top_k=10)
docs

[Document(metadata={'object': 'Communes_In_France', 'object_id': 27, 'relation': 'Instance_of', 'relation_id': 6, 'subject': 'Balleroy', 'subject_id': 26, 'timestamp': 'T_0', 'timestamp_id': 0}, page_content='"Balleroy was an instance of Communes_In_France on T_0"'),
 Document(metadata={'object': 'Communes_In_France', 'object_id': 27, 'relation': 'Instance_of', 'relation_id': 6, 'subject': 'Balleroy', 'subject_id': 26, 'timestamp': 'T_4', 'timestamp_id': 4}, page_content='"Balleroy was an instance of Communes_In_France on T_4"'),
 Document(metadata={'object': 'Communes_In_France', 'object_id': 27, 'relation': 'Instance_of', 'relation_id': 6, 'subject': 'Balleroy', 'subject_id': 26, 'timestamp': 'T_10', 'timestamp_id': 10}, page_content='"Balleroy was an instance of Communes_In_France on T_10"'),
 Document(metadata={'object': 'Communes_In_France', 'object_id': 27, 'relation': 'Instance_of', 'relation_id': 6, 'subject': 'Balleroy', 'subject_id': 26, 'timestamp': 'T_7', 'timestamp_id': 7}

---

### Adding new documents

In [7]:
from langchain.vectorstores import Chroma
import os
from langchain_community.vectorstores.utils import filter_complex_metadata
from process_embedding.custom_embedding_function import CustomEmbeddingFunction

db_directory = "result/YAGO/vector_database_build"
collection = "facts"

# Đường dẫn đến thư mục lưu trữ collection
collection_persist_directory = os.path.join(db_directory, collection)

# Khởi tạo Chroma với tham số CHÍNH XÁC như lúc tạo collection ban đầu
db = Chroma(persist_directory=collection_persist_directory, embedding_function=CustomEmbeddingFunction(llm_instance))

# Kiểm tra xem collection đã load đúng chưa
print("Số documents trước khi thêm:", db._collection.count())

# # Thêm documents mới (đảm bảo metadata khớp schema)
# new_docs = [...]
# db.add_documents(documents=new_docs)

# Kiểm tra lại
x = db.get()['ids']
numbers = sorted(int(num) for num in x)



Số documents trước khi thêm: 544212


In [8]:
missing_sequences = []
for i in range(1, len(numbers)):
    prev_num = numbers[i-1]
    current_num = numbers[i]
    gap = current_num - prev_num
    
    if gap > 1:
        start_missing = prev_num + 1
        end_missing = current_num - 1
        missing_count = end_missing - start_missing + 1
        missing_sequences.append((start_missing, end_missing, missing_count))
target_sequence = None
for start, end, count in missing_sequences:
    if count == 20:
        target_sequence = list(range(start, end + 1))
        break

if target_sequence:
    print("Dãy 20 số thiếu:", target_sequence)
else:
    print("Không tìm thấy dãy 20 số liên tiếp thiếu.")

Không tìm thấy dãy 20 số liên tiếp thiếu.
